In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg, FedAdagrad
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset
from flwr.common import ndarrays_to_parameters, NDArrays, Scalar, Context

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 20:41:48,573	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cpu
Flower 1.17.0 / PyTorch 2.6.0+cu126


## Data Loading

In [2]:
NUM_PARTITIONS = 10
BATCH_SIZE = 32


def load_datasets(partition_id: int, num_partitions: int):
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": num_partitions})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    def apply_transforms(batch):
        # Instead of passing transforms to CIFAR10(..., transform=transform)
        # we will use this function to dataset.with_transform(apply_transforms)
        # The transforms object is exactly the same
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

## Model Training/evaluation

In [3]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

## Flower Client

In [4]:
class FlowerClient(NumPyClient):
    def __init__(self, partition_id, net, trainloader, valloader):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.partition_id}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = Net().to(DEVICE)

    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

# Strategy customization

## Server-side parameter inicialization

In [5]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

In [6]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(
            params
        ),  # Pass initial model parameters
    )

    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)

In [7]:
# Create ServerApp
server = ServerApp(server_fn=server_fn)

In [8]:
# Specify the resources each of your clients need
# If set to none, by default, each client will be allocated 2x CPU and 0x GPUs
backend_config = {"client_resources": None}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_gpus": 1}}

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
(ClientAppActor pid=52280) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
(ClientAppActor pid=52280)   obj.co_lnotab,  # for < python 3.10 [not counted in args]


(ClientAppActor pid=52280) [Client 6] fit, config: {}
(ClientAppActor pid=33912) Epoch 1: train loss 0.06435737758874893, accuracy 0.23625


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=45012) [Client 0] evaluate, config: {}
(ClientAppActor pid=45012) [Client 3] fit, config: {} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(ClientAppActor pid=45012) Epoch 1: train loss 0.06423526257276535, accuracy 0.23275 [repeated 2x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=33912) [Client 5] evaluate, config: {}
(ClientAppActor pid=45012) [Client 8] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=45012) Epoch 1: train loss 0.0530581995844841, accuracy 0.37575


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 51.87s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06109349083900451
INFO :      		round 2: 0.05462989560763041
INFO :      		round 3: 0.051688401420911155
INFO :      


(ClientAppActor pid=45012) [Client 1] evaluate, config: {} [repeated 2x across cluster]


(ClientAppActor pid=45012) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead. [repeated 2x across cluster]
(ClientAppActor pid=45012)   obj.co_lnotab,  # for < python 3.10 [not counted in args] [repeated 2x across cluster]


## Comenzando una estrategia personalizada

In [9]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAdagrad strategy
    strategy = FedAdagrad(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
    )
    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
(ClientAppActor pid=42872) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
(ClientAppActor pid=42872)   obj.co_lnotab,  # for < python 3.10 [not counted in args]


(ClientAppActor pid=42872) [Client 6] fit, config: {}
(ClientAppActor pid=42872) Epoch 1: train loss 0.06425897777080536, accuracy 0.24


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=42216) [Client 5] evaluate, config: {}
(ClientAppActor pid=33204) [Client 4] fit, config: {} [repeated 2x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=42216) Epoch 1: train loss 0.1404278576374054, accuracy 0.153
(ClientAppActor pid=42216) [Client 5] evaluate, config: {}
(ClientAppActor pid=33204) [Client 8] fit, config: {}
(ClientAppActor pid=33204) [Client 1] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 48.14s
INFO :      	History (loss, distributed):
INFO :      		round 1: 6.1100668004353835
INFO :      		round 2: 1.0775171349843342
INFO :      		round 3: 0.09510739374160766
INFO :      


(ClientAppActor pid=33204) Epoch 1: train loss 0.13795095682144165, accuracy 0.1475
(ClientAppActor pid=42872) [Client 9] evaluate, config: {}


(ClientAppActor pid=33204) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead. [repeated 2x across cluster]
(ClientAppActor pid=33204)   obj.co_lnotab,  # for < python 3.10 [not counted in args] [repeated 2x across cluster]


# Evaluación de parámetros en el lado del servidor

In [10]:
# The `evaluate` function will be called by Flower after every round
def evaluate(
    server_round: int,
    parameters: NDArrays,
    config: Dict[str, Scalar],
) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    net = Net().to(DEVICE)
    _, _, testloader = load_datasets(0, NUM_PARTITIONS)
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, testloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [11]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create the FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=evaluate,  # Pass the evaluation function
    )
    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

In [12]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
INFO :      initial parameters (loss, other metrics): 0.07210723242759705, {'accuracy': 0.1013}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07210723242759705 / accuracy 0.1013


(ClientAppActor pid=40636) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
(ClientAppActor pid=40636)   obj.co_lnotab,  # for < python 3.10 [not counted in args]


(ClientAppActor pid=40636) [Client 2] fit, config: {}
(ClientAppActor pid=40636) Epoch 1: train loss 0.06473325192928314, accuracy 0.235


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (1, 0.059584119701385496, {'accuracy': 0.3043}, 19.230481600010535)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (3, 0.050359944796562195, {'accuracy': 0.4123}, 57.716322799999034)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.050359944796562195 / accuracy 0.4123
(ClientAppActor pid=40636) [Client 8] evaluate, config: {}
(ClientAppActor pid=40636) [Client 3] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=40636) Epoch 1: train loss 0.05274926498532295, accuracy 0.388 [repeated 2x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 62.76s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06096100234985352
INFO :      		round 2: 0.055256288687388105
INFO :      		round 3: 0.05234475779533387
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07210723242759705
INFO :      		round 1: 0.059584119701385496
INFO :      		round 2: 0.05353307328224182
INFO :      		round 3: 0.050359944796562195
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1013), (1, 0.3043), (2, 0.3688), (3, 0.4123)]}
INFO :      


(ClientAppActor pid=2264) [Client 1] evaluate, config: {} [repeated 2x across cluster]


(ClientAppActor pid=42288) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead. [repeated 2x across cluster]
(ClientAppActor pid=42288)   obj.co_lnotab,  # for < python 3.10 [not counted in args] [repeated 2x across cluster]


# Envío/recepción de valores arbitrarios desde/hacia los clientes

In [13]:
class FlowerClient(NumPyClient):
    def __init__(self, pid, net, trainloader, valloader):
        self.pid = pid  # partition ID of a client
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.pid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.pid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.pid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = Net().to(DEVICE)
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]
    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

Asignamos una función a la estrategia, que la invoca en cada ronda de aprendizaje federado

In [14]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,
    }
    return config

Pasar esta función a la estrategia FedAvg antes de comenzar la simulación

In [15]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=evaluate,
        on_fit_config_fn=fit_config,  # Pass the fit_config function
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.07210723242759705, {'accuracy': 0.1013}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07210723242759705 / accuracy 0.1013


(ClientAppActor pid=43908) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
(ClientAppActor pid=43908)   obj.co_lnotab,  # for < python 3.10 [not counted in args]


(ClientAppActor pid=43908) [Client 9, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


INFO :      aggregate_fit: received 3 results and 0 failures


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (3, 0.04782571606636048, {'accuracy': 0.4409}, 60.52331589999085)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.04782571606636048 / accuracy 0.4409
(ClientAppActor pid=49388) [Client 3] evaluate, config: {}
(ClientAppActor pid=49388) [Client 3, round 3] fit, config: {'server_round': 3, 'local_epochs': 2} [repeated 2x across cluster]
(ClientAppActor pid=49388) Epoch 2: train loss 0.047959666699171066, accuracy 0.43975 [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 67.56s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.061586835145950314
INFO :      		round 2: 0.053778018236160276
INFO :      		round 3: 0.04949627685546875
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07210723242759705
INFO :      		round 1: 0.06005197138786316
INFO :      		round 2: 0.051186873030662536
INFO :      		round 3: 0.04782571606636048
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1013), (1, 0.3004), (2, 0.4084), (3, 0.4409)]}
INFO :      


(ClientAppActor pid=43908) [Client 6] evaluate, config: {} [repeated 2x across cluster]


(ClientAppActor pid=52776) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead. [repeated 2x across cluster]
(ClientAppActor pid=52776)   obj.co_lnotab,  # for < python 3.10 [not counted in args] [repeated 2x across cluster]


# Escalamiento del aprendizaje federado

In [16]:
NUM_PARTITIONS = 1000

In [17]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.025,  # Train on 25 clients (each round)
        fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
        min_fit_clients=20,
        min_evaluate_clients=40,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        on_fit_config_fn=fit_config,
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 25 clients (out of 1000)
(ClientAppActor pid=50492) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
(ClientAppActor pid=50492)   obj.co_lnotab,  # for < python 3.10 [not counted in args]


(ClientAppActor pid=50492) [Client 98, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=50492) Epoch 1: train loss 0.11510894447565079, accuracy 0.1
(ClientAppActor pid=50492) Epoch 2: train loss 0.11440613120794296, accuracy 0.15
(ClientAppActor pid=50492) Epoch 3: train loss 0.11258213222026825, accuracy 0.175
(ClientAppActor pid=46036) [Client 556, round 1] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 14x across cluster]
(ClientAppActor pid=46036) Epoch 2: train loss 0.11314308643341064, accuracy 0.15 [repeated 41x across cluster]
(ClientAppActor pid=33152) [Client 932, round 1] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 9x across cluster]
(ClientAppActor pid=33152) Epoch 2: train loss 0.11385456472635269, accuracy 0.175 [repeated 27x across cluster]


INFO :      aggregate_fit: received 25 results and 0 failures
INFO :      configure_evaluate: strategy sampled 50 clients (out of 1000)


(ClientAppActor pid=17404) [Client 7] evaluate, config: {}
(ClientAppActor pid=17404) [Client 912, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=17404) Epoch 3: train loss 0.11313526332378387, accuracy 0.2 [repeated 4x across cluster]
(ClientAppActor pid=42636) [Client 637] evaluate, config: {} [repeated 15x across cluster]
(ClientAppActor pid=33152) [Client 945] evaluate, config: {} [repeated 9x across cluster]
(ClientAppActor pid=41108) [Client 49] evaluate, config: {} [repeated 11x across cluster]
(ClientAppActor pid=50492) [Client 867] evaluate, config: {} [repeated 13x across cluster]


INFO :      aggregate_evaluate: received 50 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 25 clients (out of 1000)


...

INFO :      aggregate_fit: received 25 results and 0 failures
INFO :      configure_evaluate: strategy sampled 50 clients (out of 1000)


(ClientAppActor pid=33152) [Client 116] evaluate, config: {}
(ClientAppActor pid=33152) [Client 845, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(ClientAppActor pid=33152) Epoch 3: train loss 0.10877363383769989, accuracy 0.275 [repeated 6x across cluster]
(ClientAppActor pid=38204) [Client 368] evaluate, config: {} [repeated 15x across cluster]
(ClientAppActor pid=50492) [Client 508] evaluate, config: {} [repeated 14x across cluster]
(ClientAppActor pid=32780) [Client 300] evaluate, config: {} [repeated 9x across cluster]
(ClientAppActor pid=41108) [Client 765] evaluate, config: {} [repeated 9x across cluster]


INFO :      aggregate_evaluate: received 50 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 152.11s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.2297711215019226
INFO :      		round 2: 0.2286809606552124
INFO :      		round 3: 0.22469676876068115
INFO :      


(ClientAppActor pid=42636) [Client 104] evaluate, config: {} [repeated 2x across cluster]


(ClientAppActor pid=33152) c:\Users\rafae\Codigo\federated_learning\venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead. [repeated 7x across cluster]
(ClientAppActor pid=33152)   obj.co_lnotab,  # for < python 3.10 [not counted in args] [repeated 7x across cluster]
